In [1]:
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px

C:\Users\Dennis\AppData\Local\Temp/ipykernel_11488/1855361594.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\Dennis\AppData\Local\Temp/ipykernel_11488/1855361594.py:4: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


In [2]:
# Read the airline data into pandas dataframe
spacex_df = pd.read_csv("spacex_launch_dash.csv")
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

In [3]:
# Create a dash application
app = dash.Dash(__name__)

In [4]:
# Create an app layout
app.layout = html.Div(children=[html.H1('SpaceX Launch Records Dashboard',
                                        style={'textAlign': 'center', 'color': '#503D36',
                                               'font-size': 40}),
                                # TASK 1: Add a dropdown list to enable Launch Site selection
                                # The default select value is for ALL sites
                                dcc.Dropdown(id='site-dropdown',
                                            options=[
                                                    {'label': 'All Sites', 'value': 'ALL'},
                                                    {'label': 'CCAFS LC-40','value':'CCAFS LC-40'},
                                                    {'label': 'CCAFS SLC-40','value':'CCAFS SLC-40'},
                                                    {'label': 'KSC LC-39A','value':'KSC LC-39A'},
                                                    {'label': 'VAFB SLC-4E','value':'VAFB SLC-4E'}
                                                    ],
                                            value='ALL',
                                            placeholder="Select a Launch Site here",
                                            searchable=True
                                            ),
                                html.Br(),

                                # TASK 2: Add a pie chart to show the total successful launches count for all sites
                                # If a specific launch site was selected, show the Success vs. Failed counts for the site
                                html.Div(dcc.Graph(id='success-pie-chart')),
                                html.Br(),

                                html.P("Payload range (Kg):"),
                                # TASK 3: Add a slider to select payload range
                                dcc.RangeSlider(id='payload-slider',
                                                min=0, max=10000, step=1000,
                                                marks={0: '0',
                                                       2500: '2500',
                                                       5000:'5000',
                                                       7500:'7500',
                                                       10000:'10000'},
                                                value=[min_payload, max_payload]),

                                # TASK 4: Add a scatter chart to show the correlation between payload and launch success
                                html.Div(dcc.Graph(id='success-payload-scatter-chart')),
                                ])

In [5]:
# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output

@app.callback(Output(component_id='success-pie-chart', component_property='figure'),
              Input(component_id='site-dropdown', component_property='value'))
def get_pie_chart(entered_site):
    if entered_site == 'ALL':
        fig = px.pie(spacex_df, values='class', 
                    names='Launch Site', 
                    title='Total Success Launches By Site')
        return fig
    else:
        filtered_df=spacex_df[spacex_df['Launch Site']==entered_site]
        grouped_df=filtered_df.groupby(['Launch Site','class']).size().reset_index(name='class count') 
        fig = px.pie(grouped_df, values='class count', 
                    names='class', 
                    title='Total Success Launches for site '+entered_site)
        return fig

In [6]:
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output

@app.callback(Output(component_id='success-payload-scatter-chart', component_property='figure'),
              [Input(component_id='site-dropdown', component_property='value'),
              Input(component_id='payload-slider', component_property='value')])
def get_scatter_chart(entered_site, payload_range):
    low, high=payload_range
    df_slide=spacex_df[spacex_df['Payload Mass (kg)'].between(low,high)]
    if entered_site=='ALL':
        fig=px.scatter(df_slide, x='Payload Mass (kg)', y='class', color='Booster Version Category',
        title='Success Rate by Payload and Booster Versions for All Sites')
        return fig
    
    else:
        df_select=df_slide[df_slide['Launch Site'] == entered_site]
        fig=px.scatter(df_select,x='Payload Mass (kg)', y='class', color='Booster Version Category',
        title='Success Rate by Payload and Booster Versions for site '+entered_site)
        return fig

In [ ]:
# Run the app
if __name__ == '__main__':
    app.run_server()


Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [29/Jul/2022 18:59:31] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2022 18:59:31] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2022 18:59:31] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2022 18:59:31] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2022 18:59:31] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 304 -
127.0.0.1 - - [29/Jul/2022 18:59:31] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 304 -
127.0.0.1 - - [29/Jul/2022 18:59:31] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2022 18:59:33] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2022 18:59:33] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2022 18:59:42] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [29/Jul/2022 18:59:43] "POST /_dash-

Question 3

Which attribute is used to provide available selections (such as a list of launch sites) for a Plotly DropDown input? 

options

Question 4

How can we associate the result of a callback function (like a Plotly figure) to an element defined in the application layout?

Using a unique component id

Question 5

Can we add multiple input components to a dash callback function (2 pts)?

Yes